In [199]:
from decimal import Decimal, getcontext
from numpy import inf
from math import factorial
import pandas as pd

In [200]:
def get_p0(lamb, mu, s, c, precision=6):
    getcontext().prec = precision + 6

    assert(lamb > 0), "Lambda must be greater than 0"
    assert(mu > 0), "Mu must be greater than 0"
    assert(s > 0), "Servers Lambda must be greater than 0"
    assert(c > 0), "Line capacity must be greater than 0"

    N = s + c
    sigma = Decimal(lamb) / Decimal(mu)
    p_0 = Decimal(0)

    if (s == 1) and (c != inf):
        p_0 += (1 - sigma) / (1 - sigma ** (N + 1))
    elif (s == 1) and (c == inf):
        assert sigma < 1, "System must be stable (ρ < 1)."
        
        p_0 += 1 - sigma
    elif (s != 1) and (c != inf):
        s_factorial = Decimal(factorial(s))
        for i in range(N + 1):
            if i < s:
                p_0 += (sigma ** i) / Decimal(factorial(i))
            else:
                p_0 += (sigma ** i) / ((s_factorial) * Decimal(s ** (i - s)))

        p_0 = 1 / p_0
    elif (s != 1) and (c == inf):
        assert sigma / s < 1, "System must be stable (ρ < 1)."

        for i in range(s):
            p_0 += (sigma ** i) / Decimal(factorial(i))
        
        p_0 += (sigma ** s) / (Decimal(factorial(s - 1)) * (s - sigma))

        p_0 = 1 / p_0
    else:
        print("Failed.")
    
    return p_0

In [201]:
def get_pn(lamb, mu, s, c, n, precision=6):
    getcontext().prec = precision + 6

    assert(lamb > 0), "Lambda must be greater than 0"
    assert(mu > 0), "Mu must be greater than 0"
    assert(s > 0), "Servers Lambda must be greater than 0"
    assert(c > 0), "Line capacity must be greater than 0"

    sigma = Decimal(lamb) / Decimal(mu)
    p_0 = Decimal(get_p0(lamb, mu, s, c, precision))

    if (s == 1):
        p_n = (sigma ** n) * p_0
    else:
        if n < s:
            p_n = (sigma ** n) * (p_0) / (Decimal(factorial(n)))
        else:
            p_n = (sigma ** n) * (p_0) / (Decimal(factorial(s)) * (s ** (n - s)))
    
    return p_n

In [202]:
def get_l_ef(lamb, mu, s, c, precision=6):
    getcontext().prec = precision + 6

    assert(lamb > 0), "Lambda must be greater than 0"
    assert(mu > 0), "Mu must be greater than 0"
    assert(s > 0), "Servers Lambda must be greater than 0"
    assert(c > 0), "Line capacity must be greater than 0"

    N = s + c
    sigma = Decimal(lamb) / Decimal(mu)

    if (c != inf):
        l_ef = Decimal(lamb) * (1 - Decimal(get_pn(lamb, mu, s, c, N, precision)))

    elif (c == inf):
        assert sigma / s < 1, "System must be stable (ρ < 1)."
        l_ef = Decimal(lamb)
    else:
        l_ef = -1
    return l_ef

In [203]:
def get_L(lamb, mu, s, c, precision=6):
    getcontext().prec = precision + 6

    assert(lamb > 0), "Lambda must be greater than 0"
    assert(mu > 0), "Mu must be greater than 0"
    assert(s > 0), "Servers Lambda must be greater than 0"
    assert(c > 0), "Line capacity must be greater than 0"

    # L = W * lambda

    W = Decimal(get_W(lamb, mu, s, c, precision))
    l_ef = Decimal(get_l_ef(lamb, mu, s, c, precision))
    L = W * l_ef

    return L

In [204]:
def get_Lq(lamb, mu, s, c, precision=6):
    getcontext().prec = precision + 6

    assert(lamb > 0), "Lambda must be greater than 0"
    assert(mu > 0), "Mu must be greater than 0"
    assert(s > 0), "Servers Lambda must be greater than 0"
    assert(c > 0), "Line capacity must be greater than 0"

    N = s + c
    sigma = Decimal(lamb) / Decimal(mu)

    if (s == 1) and (c != inf):
        L_q = sum((n - 1) * get_pn(lamb, mu, s, c, n, precision) for n in range(1, N + 1))
    elif (s == 1) and (c == inf):
        assert sigma < 1, "System must be stable (ρ < 1)."
        L_q = (Decimal(lamb) ** 2) / (Decimal(mu) * (Decimal(mu) - Decimal(lamb)))
    elif (s != 1) and (c != inf):
        L_q = sum((n - s) * get_pn(lamb, mu, s, c, n, precision) for n in range(s, N + 1))
    elif (s != 1) and (c == inf):
        assert sigma / s < 1, "System must be stable (ρ < 1)."
        p_0 = Decimal(get_p0(lamb, mu, s, c, precision))
        L_q =  p_0 * (sigma ** (s + 1)) / (Decimal(factorial(s - 1)) * ((s - sigma) ** 2))
    else:
        L_q = -1
    return L_q

In [205]:
def get_W(lamb, mu, s, c, precision=6):
    getcontext().prec = precision + 6

    assert(lamb > 0), "Lambda must be greater than 0"
    assert(mu > 0), "Mu must be greater than 0"
    assert(s > 0), "Servers Lambda must be greater than 0"
    assert(c > 0), "Line capacity must be greater than 0"

    W_q = Decimal(get_Wq(lamb, mu, s, c, precision))

    W = W_q + Decimal(1 / mu)

    return W

In [206]:
def get_Wq(lamb, mu, s, c, precision=6):
    getcontext().prec = precision + 6

    assert(lamb > 0), "Lambda must be greater than 0"
    assert(mu > 0), "Mu must be greater than 0"
    assert(s > 0), "Servers Lambda must be greater than 0"
    assert(c > 0), "Line capacity must be greater than 0"

    L_q = Decimal(get_Lq(lamb, mu, s, c, precision))
    l_ef = Decimal(get_l_ef(lamb, mu, s, c, precision))

    W_q = L_q / l_ef

    return W_q

In [207]:
def get_rho(lamb, mu, s, c, precision):
    getcontext().prec = precision + 6

    assert(lamb > 0), "Lambda must be greater than 0"
    assert(mu > 0), "Mu must be greater than 0"
    assert(s > 0), "Servers Lambda must be greater than 0"
    assert(c > 0), "Line capacity must be greater than 0"

    l_ef = Decimal(get_l_ef(lamb, mu, s, c, precision))

    rho = l_ef / Decimal(s * mu)

    return rho

In [208]:
def summary(lamb, mu, s, c, precision=6):
    getcontext().prec = precision + 6

    assert(lamb > 0), "Lambda must be greater than 0"
    assert(mu > 0), "Mu must be greater than 0"
    assert(s > 0), "Servers Lambda must be greater than 0"
    assert(c > 0), "Line capacity must be greater than 0"

    summary = {
        'L_ef' : round(get_l_ef(lamb, mu, s, c, precision), precision),
        'P_0' : round(get_p0(lamb, mu, s, c, precision), precision),
        'Rho' : round(get_rho(lamb, mu, s, c, precision), precision),
        'L' : round(get_L(lamb, mu, s, c, precision), precision),
        'L_q' : round(get_Lq(lamb, mu, s, c, precision), precision),
        'W' : round(get_W(lamb, mu, s, c, precision), precision),
        'W_q' : round(get_Wq(lamb, mu, s, c, precision), precision),
    }

    return summary

# Filas Infinitas S > 1

## Problema 1

In [209]:
args_1 = {
    'lamb' : 8,
    'mu' : 5,
    's' : 2,
    'c' : inf,
    'precision' : 6
}

args_2 = {
    'lamb' : 16,
    'mu' : 5,
    's' : 4,
    'c' : inf,
    'precision' : 6
}

In [210]:
summary(**args_1)

{'L_ef': Decimal('8.000000'),
 'P_0': Decimal('0.111111'),
 'Rho': Decimal('0.800000'),
 'L': Decimal('4.444444'),
 'L_q': Decimal('2.844444'),
 'W': Decimal('0.555556'),
 'W_q': Decimal('0.355556')}

In [211]:
summary(**args_2)

{'L_ef': Decimal('16.000000'),
 'P_0': Decimal('0.027303'),
 'Rho': Decimal('0.800000'),
 'L': Decimal('5.585730'),
 'L_q': Decimal('2.385730'),
 'W': Decimal('0.349108'),
 'W_q': Decimal('0.149108')}

## Problema 2

In [212]:
args_1 = {
    'lamb' : 2,
    'mu' : 3,
    's' : 1,
    'c' : inf,
    'precision' : 6
}

args_2 = {
    'lamb' : 2,
    'mu' : 3,
    's' : 2,
    'c' : inf,
    'precision' : 6
}

In [213]:
summary(**args_1)

{'L_ef': Decimal('2.000000'),
 'P_0': Decimal('0.333333'),
 'Rho': Decimal('0.666667'),
 'L': Decimal('2.000000'),
 'L_q': Decimal('1.333333'),
 'W': Decimal('1.000000'),
 'W_q': Decimal('0.666667')}

In [214]:
summary(**args_2)

{'L_ef': Decimal('2.000000'),
 'P_0': Decimal('0.500000'),
 'Rho': Decimal('0.333333'),
 'L': Decimal('0.750000'),
 'L_q': Decimal('0.083333'),
 'W': Decimal('0.375000'),
 'W_q': Decimal('0.041667')}

In [215]:
pd.DataFrame([summary(**args_1), summary(**args_2)])

,L_ef,P_0,Rho,L,L_q,W,W_q
0,2.000000,0.333333,0.666667,2.000000,1.333333,1.000000,0.666667
1,2.000000,0.500000,0.333333,0.750000,0.083333,0.375000,0.041667


# Filas Finitas S>1

## Problema 1

In [216]:
args_1 = {
    'lamb' : 20,
    'mu' : 10,
    's' : 2,
    'c' : 3,
    'precision' : 6
}

In [217]:
summary(**args_1)

{'L_ef': Decimal('16.363636'),
 'P_0': Decimal('0.090909'),
 'Rho': Decimal('0.818182'),
 'L': Decimal('2.727273'),
 'L_q': Decimal('1.090909'),
 'W': Decimal('0.166667'),
 'W_q': Decimal('0.066667')}

In [218]:
'''
a) La proporción de automóviles que llenarán el tanque en otro lado.
'''

get_pn(**args_1, n=5)

Decimal('0.181818181818')

In [219]:
'''
b) La proporción de tiempo en el que se usa una bomba.
'''

get_pn(**args_1, n=1)

Decimal('0.181818181818')

In [220]:
'''
c) La utilización proporcional de las dos bombas.
'''
# L_s = L - Lq = elementos que están siendo atendidos

(get_L(**args_1) - get_Lq(**args_1)) / args_1['s']

Decimal('0.81818181818')

In [221]:
'''
d) La probabilidad de que un automóvil que llegue no reciba servicio de inmediato, sino que se forme en la cola.
'''

print([n for n in range(args_1['s'], (args_1['s'] + args_1['c']))])
sum(get_pn(**args_1, n=n) for n in range(args_1['s'], (args_1['s'] + args_1['c'])))

[2, 3, 4]


Decimal('0.545454545455')

## Problema 2

In [ ]:
args_1 = {
    'lamb' : 2,
    'mu' : 3,
    's' : 2,
    'c' : 4,
    'precision' : 6
}

In [223]:
summary(**args_1)

{'L_ef': Decimal('1.997255'),
 'P_0': Decimal('0.500343'),
 'Rho': Decimal('0.332876'),
 'L': Decimal('0.745367'),
 'L_q': Decimal('0.079616'),
 'W': Decimal('0.373196'),
 'W_q': Decimal('0.039863')}

In [ ]:
sum(get_pn(**args_1, n=n) for n in [2,3,4,5])*8

Decimal('1.31777625258')

In [230]:
sum([get_pn(**args_1, n=n) for n in [2,3,4,5]])*2*8

Decimal('2.63555250517')

In [247]:
args_1 = {
    'lamb' : 12,
    'mu' : 60/13,
    's' : 3,
    'c' : inf,
    'precision' : 6
}

In [248]:
summary(**args_1)

{'L_ef': Decimal('12.000000'),
 'P_0': Decimal('0.034542'),
 'Rho': Decimal('0.866667'),
 'L': Decimal('7.532815'),
 'L_q': Decimal('4.932815'),
 'W': Decimal('0.627735'),
 'W_q': Decimal('0.411068')}

In [241]:
1-sum(get_pn(**args_1, n=n) for n in [0, 1])

Decimal('0.390625000000')